In [ ]:
import JupyterFormatter
JupyterFormatter.enable_autoformat();

In [ ]:
import HTTP
import JSON

In [ ]:
ALL_MEDIUMS = ["manga", "anime"];

In [ ]:
URLS = Dict(
    "fetch_media_lists" => "http://localhost:3000",
    "compress_media_lists" => "http://localhost:3001",
    "nondirectional" => "http://localhost:3002",
    "transformer_jl" => "http://localhost:3003",
    "transformer_py" => "http://localhost:3004",
    "bagofwords_jl" => "http://localhost:3005",
    "bagofwords_py" => "http://localhost:3006",
    "ensemble" => "http://localhost:3007",
);

In [ ]:
run(app, path) = HTTP.get(URLS[app] * path)
run(app, path, json) =
    HTTP.post(URLS[app] * path, [("Content-Type", "application/json")], json);

In [ ]:
function wake()
    Threads.@threads for x in collect(keys(URLS))
        run(x, "/wake")
    end
end;

In [ ]:
function get_media_list(username, source)
    responses = Dict{Any,Any}(x => nothing for x in ALL_MEDIUMS)
    Threads.@threads for medium in ALL_MEDIUMS
        r1 = run(
            "fetch_media_lists",
            "/query?username=$username&source=$source&medium=$medium",
        )
        r2 = run(
            "compress_media_lists",
            "/query?username=$username&source=$source&medium=$medium",
            String(r1.body),
        )
        responses[medium] = JSON.parse(String(r2.body))
    end
    JSON.json(responses)
end;

In [ ]:
function transformer(data)
    r_process = run("transformer_jl", "/process", data)
    input = String(r_process.body)
    responses = Dict{Any,Any}(x => nothing for x in ALL_MEDIUMS)
    Threads.@threads for m in ALL_MEDIUMS
        responses[m] = run("transformer_py", "/query?medium=$m", input)
    end
    embeddings = merge([JSON.parse(String(copy(x.body))) for x in values(responses)]...)
    r_compute = run(
        "transformer_jl",
        "/compute",
        JSON.json(Dict("payload" => JSON.parse(data), "embeddings" => embeddings)),
    )
    JSON.parse(String(r_compute.body))
end;

In [ ]:
function bagofwords(data)
    r_process = run("bagofwords_jl", "/process", data)
    process = JSON.parse(String(r_process.body))
    alphas = process["alpha"]
    input = JSON.json(process["features"])
    responses = Dict{Any,Any}(x => nothing for x in ALL_MEDIUMS)
    Threads.@threads for m in ALL_MEDIUMS
        responses[m] = run("bagofwords_py", "/query?medium=$m", input)
    end
    embeddings = merge([JSON.parse(String(copy(x.body))) for x in values(responses)]...)
    r_compute = run(
        "bagofwords_jl",
        "/compute",
        JSON.json(Dict("payload" => JSON.parse(data), "embeddings" => embeddings)),
    )
    merge(alphas, JSON.parse(String(r_compute.body)))
end;

In [ ]:
function nondirectional(data)
    r = run("nondirectional", "/query", data)
    JSON.parse(String(r.body))
end;

In [ ]:
function get_recs(username, source)
    @time data = get_media_list(username, source)
    responses =
        Dict{Any,Any}(x => nothing for x in ["bagofwords", "transformer", "nondirectional"])
    @time @sync begin
        Threads.@spawn responses["bagofwords"] = bagofwords(data)
        Threads.@spawn responses["transformer"] = transformer(data)
        Threads.@spawn responses["nondirectional"] = nondirectional(data)
    end
    alphas = merge(values(responses)...)
    inputs = JSON.json(Dict("payload" => JSON.parse(data), "alphas" => alphas))
    run("ensemble", "/query?username=$username&source=$source", inputs)
end;